**Reconhecimento de Entidade Nomeada**

Spacy

In [1]:
!pip3 install -U pip setuptools wheel
!pip3 install -U spacy[cuda102]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 19.4 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.0.1
    Uninstalling pip-23.0.1:
      Successfully uninstalled pip-23.0.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 MB 11.6 MB/s eta 0:00:00


In [2]:
!python3 -m spacy download pt_core_news_lg

2023-04-28 17:51:17.383441: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-28 17:51:20.516835: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 568.2/568.2 MB 2.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_lg')


In [3]:
import spacy 

spacy.prefer_gpu()
nlp = spacy.load('pt_core_news_lg')

In [4]:
doc = nlp('Prefeito Alexandre Kalil libera Campeonato Paulista em Belo Horizonte')

In [5]:
for ent in doc.ents:
  print(ent.label_, ent.text)

LOC Prefeito Alexandre Kalil
ORG Campeonato Paulista
LOC Belo Horizonte


**Extração de Informação**

Modelo Baseado em Regras

In [7]:
import re 

texto = "Elza Gomes da Conceição (Rio de Janeiro, 23 de junho de 1930) é uma \
        cantora e compositora brasileira."

regex = r"(.*?) \((.*?), (\d+) de (.+?) de (\d+)\) (.*) (um|uma|) (.*)."

for row in re.findall(regex, texto):
  nome, local_nascimento, dia_nascimento, mes_nascimento, ano_nascimento, _, _, profissao = row
  print(nome, ':birthPlace', local_nascimento)
  print(nome, ':birthDate', '/'.join([dia_nascimento, mes_nascimento, ano_nascimento]))
  print(nome, ':occupation', profissao)

Elza Gomes da Conceição :birthPlace Rio de Janeiro
Elza Gomes da Conceição :birthDate 23/junho/1930
Elza Gomes da Conceição :occupation cantora e compositora brasileira


Modelo Não-Supervisionado

In [8]:
!pip3 install stanza

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 802.5/802.5 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.9/240.9 kB 19.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for emoji: filename=emoji-2.2.0-py3-none-any.whl size=234926 sha256=d95f56b39806bb08ef1c4d3f63f713dc661f28de5192cf403b4ae61f889dccd7
  Stored in directory: /root/.cache/pip/wheels/02/3d/88/51a592b9ad17e7899126563698b4e3961983ebe85747228ba6
Successfully built emoji


In [9]:
import stanza
#import client module
from stanza.server import CoreNLPClient
stanza.install_corenlp()

INFO:stanza:Installing CoreNLP package into /root/stanza_corenlp


In [10]:
client = CoreNLPClient(timeout=150000000, be_quiet=True, annotators=['openie'],
endpoint='http://localhost:9001')
client.start()
import time
time.sleep(10)

INFO:stanza:Writing properties to tmp file: corenlp_server-1a2865c2fbf8400c.props
INFO:stanza:Starting server with command: java -Xmx5G -cp /root/stanza_corenlp/* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9001 -timeout 150000000 -threads 5 -maxCharLength 100000 -quiet True -serverProperties corenlp_server-1a2865c2fbf8400c.props -annotators openie -preload -outputFormat serialized


In [13]:
text = 'Albert Einstein was a German-born theoretical physicist. He developed \
	the theory of relativity.'
document = client.annotate(text, output_format='json')
triples = []
for sentence in document['sentences']:
	for triple in sentence['openie']:
		triples.append({
			'entity1': triple['subject'],
			'relation': triple['relation'],
			'entity2': triple['object']
    })

for triple in triples:
	print(triple)

{'entity1': 'Albert Einstein', 'relation': 'was', 'entity2': 'theoretical physicist'}
{'entity1': 'Einstein', 'relation': 'was', 'entity2': 'born'}
{'entity1': 'Albert Einstein', 'relation': 'was', 'entity2': 'born theoretical physicist'}
{'entity1': 'He', 'relation': 'developed', 'entity2': 'theory of relativity'}
{'entity1': 'He', 'relation': 'developed', 'entity2': 'theory'}
